In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
def create_path(relative_path):
  return f'/content/drive/My Drive/MLofi/{relative_path}'

In [71]:
import os
from music21 import *
import numpy as np

In [72]:
# Dataset used : 
# https://www.kaggle.com/zakarii/lofi-hip-hop-midi/metadata
# https://github.com/nmtremblay/lofi-samples

rootdir = "datasets/samples"

# Holds all the notes of all of our songs!
allSongs = []

for subdirs, dirs, files in os.walk(create_path(rootdir)):
    for file in files:
        currentSongPath = os.path.join(subdirs, file)
        currentSongParse = converter.parse(currentSongPath)
        currentSong = []
        
        for el in currentSongParse.recurse():   
            if isinstance(el, note.Note):
                currentSong.append(str(el.pitch))
            elif isinstance(el, chord.Chord):
                # try sorting it => sorting it doesnt make a change!
                #currCord = [str(n) for n in el.pitches]
                #currentSong.append('.'.join(sorted(currCord)))
                currentSong.append('.'.join(str(n) for n in el.pitches))
                #currentSong.append('.'.join(str(n) for n in el.normalOrder))
            # For now, all of our rests will be of the duration 1 / 8 
            # in the second part of the project, we will take their actual duration!
            elif type(el) == note.Rest:
                # el.duration.quarterLength = 0.5
                currentSong.append(str(el.fullName))
                
        allSongs.append(currentSong)
        

In [73]:
# Padding element is supposed to signify an end to one song, and the beginning of the other
paddingElement = "padding"
# paddingElement = note.Rest()
# paddingElement.duration.quarterLength = 16.0

# Currently we are adding a padding element inbetween 2 songs 
# (might work on this in the future, if it gives bad results!)
def flattenSongs(allSongs):
    allNotes = []
    
    for song in allSongs:
        #allNotes.append(str(paddingElement.fullName))
        for note in song:
            allNotes.append(note)
        allNotes.append(paddingElement)
        
    return allNotes 
    
    
allNotes = flattenSongs(allSongs)
len(allNotes)

2343

In [74]:
pathToAllNotes = create_path('output/data/allNotes.txt')
with open(pathToAllNotes, "w") as f:
  f.write(str(",".join(note for note in allNotes)))

In [75]:
uniqueNotes = set(allNotes)
mappingValuesToInt = dict()
mappingIntToValues = dict()
# LTSM works better with int values than with strings, so we need to map out notes (portrayed as strings) to ints
# (we also made a mapIntToVal, so once we need some values with our model, we can return them to their actual representation)
for i, val in enumerate(uniqueNotes):
    mappingValuesToInt[val] = i
    mappingIntToValues[i] = val
len(uniqueNotes)

451

In [76]:
# we will have a list of 20 values for an input, and the output will be a single value
# our model needs to "figure out" what the next note will be using the previous 20 notes!
seqLen = 100

# TODO: FUTURE IDEAS
# in the future, we might change test out what diffrence does it make when we use a diffrent seqLen
# for instance, seqLen = [1, 5, 10, 20, 100, ...]

# inputs and outputs for our model
X_integer_encoded = []
Y_integer_encoded = []
    
for i in range(seqLen, len(allNotes)):
    
    inputValues = allNotes[i - seqLen : i]
    outputValues = allNotes[i]
    
    X_integer_encoded.append([mappingValuesToInt[note]  for note in inputValues]);
    Y_integer_encoded.append(mappingValuesToInt[outputValues]);

X_integer_encoded = np.reshape(X_integer_encoded, (len(X_integer_encoded), seqLen, 1))

In [77]:
import tensorflow as tf
from tensorflow import keras

In [78]:
X = keras.utils.to_categorical(X_integer_encoded)
Y = keras.utils.to_categorical(Y_integer_encoded)

In [79]:
X.shape

(2293, 50, 451)

In [80]:
Y.shape

(2293, 451)

In [81]:
## HIPOTEZA #1: MODEL JE PREVIŠE JEDNOSTAVAN DA BI MOGAO DA VRŠI DOBRU PREDIKCIJU
## HIPOTEZA #2: SEQUENCE LENGTH 20 JE PREVIŠE KRATAK DA BI SE MODELOVALI KOMPLEKSNI ODNOSI U MUZICI
# Making a very basic model to test the pipeline
# model = keras.models.Sequential()
# model.add(keras.layers.Input(shape=(X.shape[1], X.shape[2])))
# model.add(keras.layers.LSTM(256))
# model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(Y.shape[1], activation=keras.activations.softmax))
# model.summary()

In [82]:
# model.compile(loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()], optimizer=keras.optimizers.Adam())

In [83]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(X.shape[1], X.shape[2])))
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))

# ovaj lstm i dropout sam ja dodao!
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.LSTM(256))
# ovaj dropout sam ja dodao!
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(512))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(Y.shape[1]))
model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [84]:
filepath = create_path("weights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5")
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [85]:
model.fit(X, Y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/30
36/36 [==============================] - ETA: 0s - loss: 5.5701
Epoch 00001: loss improved from inf to 5.57014, saving model to /content/drive/My Drive/MLofi/weights/weights-improvement-01-5.5701.hdf5
36/36 [==============================] - 13s 150ms/step - loss: 5.5701
Epoch 2/30
36/36 [==============================] - ETA: 0s - loss: 5.2805
Epoch 00002: loss improved from 5.57014 to 5.28051, saving model to /content/drive/My Drive/MLofi/weights/weights-improvement-02-5.2805.hdf5
36/36 [==============================] - 5s 148ms/step - loss: 5.2805
Epoch 3/30
36/36 [==============================] - ETA: 0s - loss: 5.2336
Epoch 00003: loss improved from 5.28051 to 5.23359, saving model to /content/drive/My Drive/MLofi/weights/weights-improvement-03-5.2336.hdf5
36/36 [==============================] - 5s 152ms/step - loss: 5.2336
Epoch 4/30
36/36 [==============================] - ETA: 0s - loss: 5.2267
Epoch 00004: loss improved from 5.23359 to 5.22670, saving model to /c